In [3]:
url_base='http://www.wrc.com/service/sasCacheApi.php?route={stub}'

In [ ]:
itinerary_stub='rallies/{rallyId}/itinerary'
startlists_stub='rallies/{rallyId}/entries'
penalties_stub='rallies/{rallyId}/penalties'
retirements_stub='rallies/{rallyId}/retirements'
stagewinners_stub='rallies/{rallyId}/stagewinners'


overall_stub='stages/{stageId}/results?rallyId={rallyId}'
split_times_stub ='stages/{stageId}/splittimes?rallyId={rallyId}'
stage_times_stage_stub='stages/{stageId}/stagetimes?rallyId={rallyId}'
stage_times_overall_stub='stages/{stageId}/results?rallyId={rallyId}'

In [9]:
meta={'rallyId':30}

In [14]:
import pandas as pd
import requests

## Itinerary

In [113]:
itinerary_json=requests.get( url_base.format(stub=itinerary_stub.format(**meta) ) ).json()
#itinerary_json

In [114]:
from pandas.io.json import json_normalize
itinerary_event = json_normalize(itinerary_json)[['eventId','itineraryId','name','priority']]
itinerary_event

,eventId,itineraryId,name,priority
0,26,31,Itinerary,0


In [70]:
itinerary_legs = json_normalize(itinerary_json,['itineraryLegs'] ).drop('itinerarySections', axis=1)
itinerary_legs

,itineraryId,itineraryLegId,legDate,name,order,startListId,status
0,31,46,2018-01-25,Thursday 25th January 2018,1,66,Running
1,31,47,2018-01-26,Friday 26th January 2018,2,68,Running
2,31,48,2018-01-27,Saturday 27th January 2018,3,70,Running
3,31,49,2018-01-28,Sunday 28th January 2018,4,73,Running


In [72]:
itinerary_sections = json_normalize(itinerary_json,
                                    ['itineraryLegs', 'itinerarySections'] ).drop(['stages','controls'],axis=1)
itinerary_sections

,itineraryLegId,itinerarySectionId,name,order
0,46,106,Section 1,1
1,47,107,Section 2,2
2,47,108,Section 3,3
3,48,109,Section 4,4
4,48,110,Section 5,5
5,48,111,Section 6,6
6,49,112,Section 7,7
7,49,113,Section 8,8


In [115]:
itinerary_stages=json_normalize(itinerary_json,['itineraryLegs', 'itinerarySections','stages'] )
itinerary_stages.head()

,code,distance,eventId,name,number,stageId,stageType,status,timingPrecision
0,SS1,36.69,26,Thoard - Sisteron,1,289,SpecialStage,Completed,Tenth
1,SS2,25.49,26,Bayons - Breziers 1,2,287,SpecialStage,Completed,Tenth
2,SS3,26.72,26,Vitrolles - Oze 1,3,286,SpecialStage,Completed,Tenth
3,SS4,30.54,26,Roussieux - Eygalayes 1,4,285,SpecialStage,Completed,Tenth
4,SS5,15.18,26,Vaumeilh - Claret 1,5,284,SpecialStage,Completed,Tenth


In [86]:
meta['stages']=itinerary_stages['stageId'].tolist()

In [54]:
itinerary_controls=json_normalize(itinerary_json,['itineraryLegs', 'itinerarySections','controls'] )
itinerary_controls['stageId'] = itinerary_controls['stageId'].fillna(0).astype(int)
itinerary_controls.head()

,code,controlId,distance,eventId,firstCarDueDateTime,firstCarDueDateTimeLocal,location,stageId,status,targetDuration,targetDurationMs,timingPrecision,type
0,TC0,1066,NaN,26,2018-01-25T16:50:00,2018-01-25T17:50:00+01:00,Monaco - Depart Day 1,0,Completed,00:00:00,0,Minute,TimeControl
1,TC1,1133,192.71,26,2018-01-25T20:40:00,2018-01-25T21:40:00+01:00,Thoard,289,Completed,03:50:00,13800000,Minute,TimeControl
2,SS1,1134,36.69,26,2018-01-25T20:43:00,2018-01-25T21:43:00+01:00,Thoard - Sisteron,289,Completed,00:03:00,180000,Minute,StageStart
3,SF1,1135,NaN,26,None,None,Thoard - Sisteron,289,Completed,00:00:00,0,Tenth,FlyingFinish
4,TC2,1123,30.71,26,2018-01-25T21:48:00,2018-01-25T22:48:00+01:00,Bayons,287,Completed,01:05:00,3900000,Minute,TimeControl


## Startlists

In [116]:
startlists_json=requests.get( url_base.format(stub=startlists_stub.format(**meta) ) ).json()
#startlists_json

In [117]:
startlists = json_normalize(startlists_json).drop('eventClasses', axis=1)
startlists.head()

,codriver.abbvName,codriver.code,codriver.country.countryId,codriver.country.iso2,codriver.country.iso3,codriver.country.name,codriver.countryId,codriver.firstName,codriver.fullName,codriver.lastName,...,groupId,identifier,manufacturer.logoFilename,manufacturer.manufacturerId,manufacturer.name,manufacturerId,priority,status,tyreManufacturer,vehicleModel
0,J. INGRASSIA,ING,76,FR,FRA,France,76,Julien,Julien INGRASSIA,INGRASSIA,...,10,1,ford,26,Ford,26,P1,Entry,Michelin,FIESTA WRC
1,D. BARRITT,BAR,235,GB,GBR,United Kingdom of Great Britain and Northern I...,235,Daniel,Daniel BARRITT,BARRITT,...,10,2,ford,26,Ford,26,P1,Entry,Michelin,FIESTA WRC
2,X. PANSERI,PAN,76,FR,FRA,France,76,Xavier,Xavier PANSERI,PANSERI,...,10,3,ford,26,Ford,26,P1,Entry,Michelin,FIESTA WRC
3,A. JAEGER,JAE,166,NO,NOR,Norway,166,Anders,Anders JAEGER,JAEGER,...,10,4,hyundai,33,Hyundai,33,P1,Rejoined,Michelin,i20 WRC
4,N. GILSOUL,GIL,22,BE,BEL,Belgium,22,Nicolas,Nicolas GILSOUL,GILSOUL,...,10,5,hyundai,33,Hyundai,33,P1,Entry,Michelin,i20 WRC


In [78]:
startlist_classes = json_normalize(startlists_json,['eventClasses'], 'entryId' )
startlist_classes.head()

,eventClassId,eventId,name,entryId
0,103,26,RC1,1483
1,103,26,RC1,1484
2,103,26,RC1,1485
3,103,26,RC1,1486
4,103,26,RC1,1487


## Penalties

In [118]:
penalties_json=requests.get( url_base.format(stub=penalties_stub.format(**meta) ) ).json()
#penalties_json

In [81]:
penalties = json_normalize(penalties_json)
penalties.head()

,controlId,entryId,penaltyDuration,penaltyDurationMs,penaltyId,reason
0,1123,1496,PT40S,40000,38,4 MN LATE
1,1123,1493,PT10S,10000,39,1 MN LATE
2,1123,1524,PT30S,30000,40,3 MN LATE
3,1123,1516,PT20S,20000,41,2 MN LATE
4,1123,1517,PT30S,30000,42,3 MN LATE


## Retirements

In [119]:
retirements_json=requests.get( url_base.format(stub=retirements_stub.format(**meta) ) ).json()
#retirements_json

In [83]:
retirements = json_normalize(retirements_json)
retirements.head()

,controlId,entryId,reason,retirementDateTime,retirementDateTimeLocal,retirementId
0,1135,1518,Mechanical,2018-01-25T21:40:00Z,2018-01-25T22:40:00+01:00,17
1,1135,1538,Off Road,2018-01-25T22:00:00Z,2018-01-25T23:00:00+01:00,18
2,1135,1546,Mechanical,2018-01-25T22:05:00Z,2018-01-25T23:05:00+01:00,19
3,1123,1528,Mechanical,2018-01-25T23:15:00Z,2018-01-26T00:15:00+01:00,20
4,1123,1534,Mechanical,2018-01-25T23:15:00Z,2018-01-26T00:15:00+01:00,21


## Stagewinners

In [120]:
stagewinners_json=requests.get( url_base.format(stub=stagewinners_stub.format(**meta) ) ).json()
#stagewinners_json

In [85]:
stagewinners = json_normalize(stagewinners_json)
stagewinners.head()

,elapsedDuration,elapsedDurationMs,entryId,stageId,stageName
0,00:14:53.2000000,893200,1483,287,Bayons - Breziers 1
1,00:23:16.6000000,1396600,1483,289,Thoard - Sisteron
2,00:08:42.6000000,522600,1484,284,Vaumeilh - Claret 1
3,00:18:25.3000000,1105300,1483,285,Roussieux - Eygalayes 1
4,00:16:32.3000000,992300,1490,286,Vitrolles - Oze 1


## Overall

In [90]:
meta2=meta
stage_overall = pd.DataFrame()
for stageId in meta['stages']:
    meta2['stageId']=stageId
    _stage_overall_json=requests.get( url_base.format(stub=overall_stub.format(**meta2) ) ).json()
    _stage_overall = json_normalize(stage_overall_json)
    _stage_overall['stageId'] = stageId
    stage_overall = pd.concat([stage_overall, _stage_overall])
stage_overall.head()

,diffFirst,diffFirstMs,diffPrev,diffPrevMs,entryId,penaltyTime,penaltyTimeMs,position,stageTime,stageTimeMs,totalTime,totalTimeMs,stageId
0,PT0S,0,PT0S,0,1483,PT0S,0,1,PT38M9.8S,2289800,PT38M9.8S,2289800,289
1,PT17.3S,17300,PT17.3S,17300,1486,PT0S,0,2,PT38M27.1S,2307100,PT38M27.1S,2307100,289
2,PT37.4S,37400,PT20.1S,20100,1491,PT0S,0,3,PT38M47.2S,2327200,PT38M47.2S,2327200,289
3,PT42.4S,42400,PT5S,5000,1490,PT0S,0,4,PT38M52.2S,2332200,PT38M52.2S,2332200,289
4,PT55.4S,55400,PT13S,13000,1489,PT0S,0,5,PT39M5.2S,2345200,PT39M5.2S,2345200,289


## Split Times

In [122]:
_stage_split_times_json
_stage_split_times.head()

,elapsedDuration,elapsedDurationMs,entryId,splitDateTime,splitDateTimeLocal,splitPointId,splitPointTimeId,startDateTime,startDateTimeLocal,stageId
0,PT4M47.113S,287113,1486,2018-01-28T11:22:47.113Z,2018-01-28T12:22:47.113+01:00,578,15848,2018-01-28T11:18:00,2018-01-28T12:18:00+01:00,288
1,PT5M22.977S,322977,1502,2018-01-28T11:27:22.977Z,2018-01-28T12:27:22.977+01:00,578,15850,2018-01-28T11:22:00,2018-01-28T12:22:00+01:00,288
2,PT4M56.194S,296194,1499,2018-01-28T11:30:56.194Z,2018-01-28T12:30:56.194+01:00,578,15851,2018-01-28T11:26:00,2018-01-28T12:26:00+01:00,288
3,PT4M49.414S,289414,1493,2018-01-28T11:34:49.414Z,2018-01-28T12:34:49.414+01:00,578,15854,2018-01-28T11:30:00,2018-01-28T12:30:00+01:00,288
4,PT4M45.104S,285104,1487,2018-01-28T11:42:45.104Z,2018-01-28T12:42:45.104+01:00,578,15856,2018-01-28T11:38:00,2018-01-28T12:38:00+01:00,288


In [103]:
split_times=pd.DataFrame()
for stageId in meta['stages']:
    meta2['stageId']=stageId
    _stage_split_times_json=requests.get( url_base.format(stub=split_times_stub.format(**meta2) ) ).json()
    _stage_split_times = json_normalize(_stage_split_times_json)
    _stage_split_times['stageId'] = stageId
    split_times = pd.concat([split_times, _stage_split_times])
split_times.head()

,elapsedDuration,elapsedDurationMs,entryId,splitDateTime,splitDateTimeLocal,splitPointId,splitPointTimeId,startDateTime,startDateTimeLocal,stageId
0,PT23M18.17S,1398170,1487,2018-01-25T21:08:18.17Z,2018-01-25T22:08:18.17+01:00,577,13338,2018-01-25T20:45:00,2018-01-25T21:45:00+01:00,289
1,PT19M47.803S,1187803,1489,2018-01-25T21:08:47.803Z,2018-01-25T22:08:47.803+01:00,577,13339,2018-01-25T20:49:00,2018-01-25T21:49:00+01:00,289
2,PT19M37.267S,1177267,1490,2018-01-25T21:06:37.267Z,2018-01-25T22:06:37.267+01:00,577,13341,2018-01-25T20:47:00,2018-01-25T21:47:00+01:00,289
3,PT19M20.088S,1160088,1488,2018-01-25T21:12:20.088Z,2018-01-25T22:12:20.088+01:00,577,13349,2018-01-25T20:53:00,2018-01-25T21:53:00+01:00,289
4,PT22M42.864S,1362864,1484,2018-01-25T21:13:42.864Z,2018-01-25T22:13:42.864+01:00,577,13352,2018-01-25T20:51:00,2018-01-25T21:51:00+01:00,289


## Stage Times - Stage

In [111]:
stage_times_stage=pd.DataFrame()
for stageId in meta['stages']:
    meta2['stageId']=stageId
    _stage_times_stage_json=requests.get( url_base.format(stub=stage_times_stage_stub.format(**meta2) ) ).json()
    _stage_times_stage = json_normalize(_stage_times_stage_json)
    _stage_times_stage['stageId'] = stageId
    stage_times_stage = pd.concat([stage_times_stage, _stage_times_stage])
stage_times_stage.head()

,diffFirst,diffFirstMs,diffPrev,diffPrevMs,elapsedDuration,elapsedDurationMs,entryId,position,source,stageId,stageTimeId,status
0,00:00:00,0.0,00:00:00,0.0,00:23:16.6000000,1396600.0,1483,1,Default,289,6639,Completed
1,00:00:07.7000000,7700.0,00:00:07.7000000,7700.0,00:23:24.3000000,1404300.0,1486,2,Default,289,6657,Completed
2,00:00:19.4000000,19400.0,00:00:11.7000000,11700.0,00:23:36,1416000.0,1491,3,Assessed,289,6655,Completed
3,00:00:19.7000000,19700.0,00:00:00.3000000,300.0,00:23:36.3000000,1416300.0,1488,4,Default,289,6649,Completed
4,00:00:24.6000000,24600.0,00:00:04.9000000,4900.0,00:23:41.2000000,1421200.0,1493,5,Default,289,6653,Completed


## Stage Times - Overall

In [112]:
stage_times_overall=pd.DataFrame()
for stageId in meta['stages']:
    meta2['stageId']=stageId
    _stage_times_overall_json=requests.get( url_base.format(stub=stage_times_overall_stub.format(**meta2) ) ).json()
    _stage_times_overall = json_normalize(_stage_times_overall_json)
    _stage_times_overall['stageId'] = stageId
    stage_times_overall = pd.concat([stage_times_overall, _stage_times_overall])
stage_times_overall.head()

,diffFirst,diffFirstMs,diffPrev,diffPrevMs,entryId,penaltyTime,penaltyTimeMs,position,stageTime,stageTimeMs,totalTime,totalTimeMs,stageId
0,PT0S,0,PT0S,0,1483,PT0S,0,1,PT23M16.6S,1396600,PT23M16.6S,1396600,289
1,PT7.7S,7700,PT7.7S,7700,1486,PT0S,0,2,PT23M24.3S,1404300,PT23M24.3S,1404300,289
2,PT19.4S,19400,PT11.7S,11700,1491,PT0S,0,3,PT23M36S,1416000,PT23M36S,1416000,289
3,PT24.6S,24600,PT5.2S,5200,1493,PT0S,0,4,PT23M41.2S,1421200,PT23M41.2S,1421200,289
4,PT37.9S,37900,PT13.3S,13300,1490,PT0S,0,5,PT23M54.5S,1434500,PT23M54.5S,1434500,289
